In [ ]:
import phik
from phik.report import plot_correlation_matrix
from tqdm import tqdm

In [ ]:
from phik.binning import create_correlation_overview_table, bin_data
from phik.bivariate import phik_from_chi2
from phik.statistics import estimate_simple_ndof


In [ ]:
import itertools
from scipy.stats import power_divergence


In [ ]:
from sdgym import load_dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from sklearn.model_selection import train_test_split

In [ ]:
import xgboost as xgb

In [ ]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch

from pgmpy.estimators import HillClimbSearch, BicScore, ExhaustiveSearch

In [ ]:
from pgmpy.estimators import BayesianEstimator
from pgmpy.sampling import BayesianModelSampling


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data, categorical_columns, ordinal_columns = load_dataset('alarm')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)
df.columns = [str(i) for i in df.columns]

In [ ]:

# learn graph structure 
est = TreeSearch(df, root_node=df.columns[0])
best_model = est.estimate(estimator_type="tan", class_node='1')

In [ ]:
if False:
    # alternative graph structure 
    est2 = TreeSearch(df, root_node=df.columns[0])
    dag2 = est2.estimate(estimator_type="chow-liu")

In [ ]:
if False:
    est = HillClimbSearch(df)
    best_model =est.estimate() # start_dag=dag)

In [ ]:
nx.draw(best_model, with_labels=True, arrowsize=30, node_size=800, alpha=0.3, font_weight='bold')
plt.show()

In [ ]:
edges = best_model.edges()

In [ ]:
edges

In [ ]:

# there are many choices of parametrization, here is one example
model = BayesianModel(best_model.edges())
model.fit(df, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=0.1)


In [ ]:

np.random.seed(0)

# sample data from BN
inference = BayesianModelSampling(model)
df_data = inference.forward_sample(size=250000, return_type='dataframe')

df_data.columns = [int(c) for c in df_data.columns]

X = df_data[sorted(df_data.columns)].values

#inference.topological_order

In [ ]:

def chisquare(observed, expected, correction=True, lambda_=None):
    observed = np.asarray(observed)
    if np.any(observed < 0):
        raise ValueError("All values in `observed` must be nonnegative.")
    if observed.size == 0:
        raise ValueError("No data; `observed` has size 0.")

    expected = np.asarray(expected)
    
    if np.any(expected == 0):
        # Include one of the positions where expected is zero in
        # the exception message.
        zeropos = list(zip(*np.nonzero(expected == 0)))[0]
        raise ValueError("The internally computed table of expected "
                         "frequencies has a zero element at %s." % (zeropos,))

    # The degrees of freedom
    dof = expected.size - sum(expected.shape) + expected.ndim - 1

    if dof == 0:
        # Degenerate case; this occurs when `observed` is 1D (or, more
        # generally, when it has only one nontrivial dimension).  In this
        # case, we also have observed == expected, so chi2 is 0.
        chi2 = 0.0
        p = 1.0
    else:
        if dof == 1 and correction:
            # Adjust `observed` according to Yates' correction for continuity.
            observed = observed + 0.5 * np.sign(expected - observed)

        chi2, p = power_divergence(observed, expected,
                                   ddof=observed.size - 1 - dof, axis=None,
                                   lambda_=lambda_)

    return chi2

def phik_from_hist2d(observed:np.ndarray, expected:np.ndarray, noise_correction:bool=True) -> float:
    """
    correlation coefficient of bivariate gaussian derived from chi2-value
    
    Chi2-value gets converted into correlation coefficient of bivariate gauss
    with correlation value rho, assuming giving binning and number of records. 
    Correlation coefficient value is between 0 and 1.

    Bivariate gaussian's range is set to [-5,5] by construction.

    :param observed: 2d-array observed values
    :param bool noise_correction: apply noise correction in phik calculation
    :returns float: correlation coefficient phik
    """

    # chi2 contingency test
    chi2 = chisquare(observed, expected, lambda_='pearson')

    # noise pedestal 
    endof = estimate_simple_ndof(observed) if noise_correction else 0
    pedestal = endof
    if pedestal < 0:
        pedestal = 0

    # phik calculation adds noise pedestal to theoretical chi2
    return phik_from_chi2(chi2, observed.sum(), *observed.shape, pedestal=pedestal)




In [ ]:
cols = [6, 34]

In [ ]:
hist = df_data[cols].hist2d(interval_cols=[])
expected = hist.values

In [ ]:
df = pd.DataFrame(data)

In [ ]:
hist = df[cols].hist2d(interval_cols=[])
observed = hist.values

In [ ]:
expected = expected * (np.sum(observed) / np.sum(expected))


In [ ]:
expected, observed

In [ ]:
phik_from_hist2d(observed, expected)

In [ ]:
phik_list = []

for co in tqdm(itertools.combinations_with_replacement(df.columns.values, 2)):
    c0, c1 = co
    if co[0]==co[1]:
        phik_list.append((c0, c1, 1.))
        continue
    hist = df_data[list(co)].hist2d(interval_cols=[])
    expected = hist.values
    hist = df[list(co)].hist2d(interval_cols=[])
    observed = hist.values
    expected = expected * (np.sum(observed) / np.sum(expected))
    phik = phik_from_hist2d(observed, expected)
    phik_list.append((c0, c1, phik))

In [ ]:
phik_overview = create_correlation_overview_table(phik_list)

# restore column order
phik_overview = phik_overview.reindex(columns=df.columns)
phik_overview = phik_overview.reindex(index=df.columns)


In [ ]:
phik_overview

In [ ]:
plot_correlation_matrix(phik_overview.values, x_labels=phik_overview.columns, y_labels=phik_overview.index, 
                        vmin=0, vmax=1, color_map='Blues', title=r'correlation $\phi_K$', fontsize_factor=0.75,
                        figsize=(14,11))
plt.tight_layout()